Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [405]:
import logging
from itertools import combinations
import pandas as pd
import numpy as np
import geopy.distance
import networkx as nx
from icecream import ic
import random
from dataclasses import dataclass
from tqdm.auto import tqdm

logging.basicConfig(level=logging.DEBUG)

In [406]:
CITIES = pd.read_csv('cities/china.csv', header=None, names=['name', 'lat', 'lon'])
CITIES.head()

,name,lat,lon
0,Acheng,45.540000,126.960000
1,Aksu,41.150000,80.250000
2,Alaer,40.515556,81.263611
3,Altay,47.840000,88.130000
4,Anbu,23.460000,116.680000


In [407]:
DIST_MATRIX= np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geopy.distance.geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km

## Lab2 - TSP

https://www.wolframcloud.com/obj/giovanni.squillero/Published/Lab2-tsp.nb

In [408]:
# Cycle detection function for edge list
def cyclic(edges):
    G = nx.Graph()
    G.add_edges_from(edges)
    try:
        nx.find_cycle(G)
        return True
    except:
        return False

def valid(tsp):
    assert tsp[0] == tsp[-1]
    assert set(tsp) == set(range(len(CITIES)))

def tsp_cost(individual):
    tsp = individual.genome  # Access the genome from the Individual
    assert tsp[0] == tsp[-1]
    assert set(tsp) == set(range(len(CITIES)))

    tot_cost = 0
    for c1, c2 in zip(tsp, tsp[1:]):
        tot_cost += DIST_MATRIX[c1, c2]
    return tot_cost

def fitness(individual):
    return -tsp_cost(individual)  # Pass Individual to tsp_cost

@dataclass
class Individual:
    genome: np.ndarray
    fitness: float = None

## First Greedy Algorithm

In [ ]:
def first_greedy_solution():
    # greedy algorithm: best choice now, not think about far away
    # is be in a city and move to nearest one
    visited = np.full(len(CITIES), False)
    dist = DIST_MATRIX.copy()
    city = random.choice(range(len(CITIES)))
    visited[city] = True
    tsp=[int(city)]
    while not np.all(visited):
        dist[:, city] = np.inf
        closest = np.argmin(dist[city])
        logging.debug(
            f"step: {CITIES.at[city,'name']} -> {CITIES.at[closest,'name']} ({DIST_MATRIX[city,closest]:.2f}km)"
        )
        visited[closest] = True
        city = closest
        tsp.append(int(city))
    logging.debug(
        f"step: {CITIES.at[tsp[-1],'name']} -> {CITIES.at[tsp[0],'name']} ({DIST_MATRIX[tsp[-1],tsp[0]]:.2f}km)"
    )
    tsp.append(tsp[0])

    individual = Individual(np.array(tsp))
    ic(fitness(individual))
    return tsp

def create_population1(pop_size):
    population = []
    for _ in range(pop_size):
        tsp = first_greedy_solution()
        individual = Individual(np.array(tsp))
        population.append(individual)
    return population


## Second Greedy Algorithm

In [411]:
segments = [
        ({c1, c2}, float(DIST_MATRIX[c1, c2])) for c1, c2 in combinations(range(len(CITIES)), 2)
    ]
segments.sort(key=lambda e: e[1])

def second_greedy_solution(start_segment, segments):
    
    visited = set()
    edges = set()
    counts = {i: 0 for i in range(len(CITIES))}
    index = 0

    # add first element and visit it
    a, b=start_segment[0]
    counts[a]+=1
    counts[b]+=1
    visited|=start_segment[0]
    edges|={tuple(start_segment[0])}

    while index<len(segments):
        nxt=segments[index]
        index+=1
        if not cyclic(edges | {tuple(nxt[0])}) and all(counts[city]<2 for city in nxt[0]):
            a, b=nxt[0]
            counts[a]+=1
            counts[b]+=1
            visited|=nxt[0]
            edges|={tuple(nxt[0])}

    # print(edges)
    # Attempt to close the cycle if exactly two unique cities are found
    lonely = [CITIES.iloc[i].name for i, count in counts.items() if count == 1]
    if len(lonely) == 2:
        closing = (lonely[0], lonely[1])
        edges.add(closing)
        counts[lonely[0]] += 1
        counts[lonely[1]] += 1
    else:
        print("Error: Impossible to close cycle.")

    # build tsp tour
    start_city = lonely[0]
    tour = [start_city]
    neighbors = {i: [] for i in range(len(CITIES))}
    for a, b in edges:
        neighbors[a].append(b)
        neighbors[b].append(a)

    # Perform a traversal to complete the TSP route
    current_city = start_city
    previous_city = None
    while True:
        next_city = next(city for city in neighbors[current_city] if city != previous_city)
        tour.append(next_city)
        if next_city == start_city:
            break
        previous_city = current_city
        current_city = next_city
        
    ic(fitness(Individual(np.array(tour))))
    return tour

    
def create_population(pop_size):  
    sorted_segments=segments
    population=[]
    start_segment=segments[0]
    for i in range(pop_size):
        sorted_segments = [s for s in sorted_segments if s != start_segment]
        sol=second_greedy_solution(start_segment, sorted_segments)
        individual = Individual(np.array(sol))
        population.append(individual)
        # start_segment=segments[i+1]
        start_segment = random.choice(sorted_segments)

    return population


In [ ]:
def parent_selection(population):
    #tournament selection
    candidates=sorted(np.random.choice(population, 2), 
                  key=lambda e : e.fitness,
                  reverse=True)
    return candidates[0]    #champion

def inver_over_crossover(parent1, parent2, crossover_prob=0.5):
    # Start with a copy of parent1's genome for the offspring
    tsp = parent1.genome.copy()
    num_cities = len(tsp) - 1  # Exclude the final return to the starting city
    
    # Iterate through each city in the tour (excluding return city)
    for i in range(num_cities):
        current_city = tsp[i]
        
        # Decide the target city for inversion
        if random.random() < crossover_prob:
            # Choose target from the other parent if within crossover probability
            target_city = parent2.genome[random.randint(0, num_cities - 1)]
        else:
            # Otherwise, choose a random city from the current tour
            target_city = random.choice([city for city in tsp if city != current_city])
        
        # Get the indices of the current city and the target city
        current_city_idx = np.where(tsp == current_city)[0][0]
        target_city_idx = np.where(tsp == target_city)[0][0]
        
        # Perform inversion between current_city and target_city
        if current_city_idx < target_city_idx:
            tsp[current_city_idx:target_city_idx + 1] = tsp[current_city_idx:target_city_idx + 1][::-1]
        else:
            tsp[target_city_idx:current_city_idx + 1] = tsp[target_city_idx:current_city_idx + 1][::-1]
    
    # Close the loop by making the last city return to the first city
    tsp = np.append(tsp[:-1], tsp[0])
    
    # Return a new Individual with the generated genome
    return Individual(genome=tsp)

def edge_recombination_crossover(parent1, parent2):
    # Create the neighborhood graph by combining the edges of both parents
    neighbors = {i: set() for i in parent1.genome[:-1]}
    for p in [parent1.genome, parent2.genome]:
        for i in range(len(p) - 1):
            neighbors[p[i]].add(p[i + 1])
            neighbors[p[i + 1]].add(p[i])
        neighbors[p[0]].add(p[-2])
        neighbors[p[-2]].add(p[0])
    
    # Builds a child from a random city
    start = parent1.genome[0]
    child = [start]
    
    while len(child) < len(parent1.genome) - 1:
        current = child[-1]
        possible_neighbors = [n for n in neighbors[current] if n not in child]
        
        if possible_neighbors:
            next_city = min(possible_neighbors, key=lambda n: len(neighbors[n]))
        else:
            remaining_cities = [n for n in range(len(parent1.genome) - 1) if n not in child]
            next_city = random.choice(remaining_cities)
        
        child.append(next_city)

    # Complete cycle
    child.append(child[0])
    return Individual(np.array(child))

def inver_mutation(individual):
    """
    Apply Inver Mutation to an individual.
    Picks two random indices and flips the segment between them.
    """
    genome = individual.genome.copy()
    size = len(genome) - 1  # Exclude the last city to close the loop

    idx1, idx2 = random.sample(range(1, size), 2)  

    if idx1 > idx2:
        idx1, idx2 = idx2, idx1

    # Invert the segment
    genome[idx1:idx2 + 1] = genome[idx1:idx2 + 1][::-1]

    return Individual(genome=genome)

def inversion_mutation(individual):
    """
    Apply Inversion Mutation to an individual.
    Chooses a random segment and reverses it.
    """
    genome = individual.genome.copy()
    size = len(genome) - 1  

    idx1, idx2 = sorted(random.sample(range(1, size), 2)) 

    # Reverse the segment
    genome[idx1:idx2 + 1] = genome[idx1:idx2 + 1][::-1]
    genome[-1] = genome[0]
    return Individual(genome=genome)

In [413]:
POPULATION_SIZE = 10
population=(create_population1(POPULATION_SIZE))
for i in population:
    i.fitness=fitness(i)

DEBUG:root:step: Leizhou -> Dongli (29.35km)
DEBUG:root:step: Dongli -> Zhanjiang (39.87km)
DEBUG:root:step: Zhanjiang -> Wuchuan (47.32km)
DEBUG:root:step: Wuchuan -> Huazhou (25.92km)
DEBUG:root:step: Huazhou -> Lianjiang (36.08km)
DEBUG:root:step: Lianjiang -> Gaozhou (60.83km)
DEBUG:root:step: Gaozhou -> Maoming (3.29km)
DEBUG:root:step: Maoming -> Beiliu (77.25km)
DEBUG:root:step: Beiliu -> Cenxi (71.65km)
DEBUG:root:step: Cenxi -> Luoding (59.07km)
DEBUG:root:step: Luoding -> Yunfu (51.30km)
DEBUG:root:step: Yunfu -> Gaoyao (44.09km)
DEBUG:root:step: Gaoyao -> Zhaoqing (2.13km)
DEBUG:root:step: Zhaoqing -> Sihui (40.92km)
DEBUG:root:step: Sihui -> Xinan (29.06km)
DEBUG:root:step: Xinan -> Foshan (28.21km)
DEBUG:root:step: Foshan -> Guangzhou (16.64km)
DEBUG:root:step: Guangzhou -> Huangpu (17.55km)
DEBUG:root:step: Huangpu -> Shiqiao (18.76km)
DEBUG:root:step: Shiqiao -> Daliang (15.81km)
DEBUG:root:step: Daliang -> Xiaolan (7.32km)
DEBUG:root:step: Xiaolan -> Zhongshan (28.60km)

In [ ]:
OFFSPRING_SIZE = 4  

MAX_GENERATION=10000
best_fitness_per_generation = []

for g in tqdm(range(MAX_GENERATION)):
    offspring=list()
    #HYPERMODERN
    for  _ in range(OFFSPRING_SIZE):
        if np.random.random()<0.7: #mutation probability
            if(g<MAX_GENERATION/2):
                p=parent_selection(population)
                o=inversion_mutation(p)
            else:
                p=parent_selection(population)
                o=inver_mutation(p)
        else:
            i1=parent_selection(population)
            i2=parent_selection(population)
            o=edge_recombination_crossover(i1, i2)
        offspring.append(o)

    for i in offspring: 
        i.fitness=fitness(i)

    population.extend(offspring)    # add them to population, sort and then cut, to come back to the original size
    population.sort(key=lambda i: i.fitness, reverse=True)
    population=population[:POPULATION_SIZE]     # if i don't cut, poopulation growing-> selective pressure increase
    best_fitness_per_generation.append(population[0].fitness)

print("Best Fitness:", population[0].fitness)
print("best path:", population[0].genome)

  0%|          | 0/10000 [00:00<?, ?it/s]

Best Fitness: -60593.96724169386
best path: [290 101 198 397 396 173 414 114 252 264 285 508 677  93 289  46 157 513
 328 384 419 212  18 343 543 138 108 671 492 519  95 327  12 287 329 616
 142  87 650 545 506 538 540  36 126  81 105 711 557 430  79 421 360 695
 452 307 306 424 308  55 561  11 148 255 438 265 254 712 460 193 324 620
 429 520 217 213 534 606 151 243 635 701 657 293 404 378 578  58 579 238
  97 548 257 425 476 524 302  54 408 686 248 273 527  85 676 281 355 655
 417 625 546  68  10 541 369 190 125 393  73 529 247 109  77 631 462  57
 581 710 351 242 311 276 663 111 654 714 332 140 184 177 487 480 679  74
 415 127 491 341 224 374 253 567 669 568 463 132 131 416  50 136 409 432
  20 453 278 440 500 398 587 357 693 689 363 659 472 407 250 251 583 486
 166 651 334 141 286 464 582 222 456 246 479  98 320 270 195 400 145 691
 621 241 271 720  64 316  51 437 200 593 485 319 323 364 656 665  47 280
 592 717 331 590 428 353  53 392 670 348 208  65 594  91 223 121 225 577
 181 59